# Library

In [19]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from datetime import datetime

# fold path

In [37]:
ods_fold = 'D:/data/big2/ODS'
fold = 'D:/data/big2'

# [로드] 등록&제원 정보 2022.12.28 기준 

In [38]:
# about 8m 18s
df = pd.DataFrame()
for i in tqdm(range(1, 6)):
    file_name = f'[ODS][G{i}]등록정보&제원정보(말소제거)(2022.12.28).csv'
    temp = pd.read_csv(os.path.join(ods_fold, file_name), encoding='cp949', low_memory=False)
    temp = temp.sort_values(['최초등록일자', '배출가스검사유효일자'], ascending=[False, False]).drop_duplicates('차대번호').reset_index(drop=True)
    df = pd.concat([df, temp], ignore_index=True)

 40%|████      | 2/5 [04:21<07:15, 145.06s/it]

In [ ]:
df.info()

In [ ]:
df.shape

(25404385, 49)

# [로드] 본거지법정동코드

In [ ]:
file_name = '법정동코드_2022.10.14.xlsx'
code = pd.read_excel(os.path.join(fold, file_name))
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46340 entries, 0 to 46339
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   법정동코드     46340 non-null  int64  
 1   시도        46340 non-null  object 
 2   시구군       46317 non-null  object 
 3   읍면동       45847 non-null  object 
 4   리         33682 non-null  object 
 5   지역3       0 non-null      float64
 6   폐지여부      46340 non-null  object 
 7   대기관리권역    46340 non-null  object 
 8   대기관리권역YN  46334 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 3.2+ MB


# [merge] 법정동코드 기준

In [ ]:
# '본거지법정도코드'열 데이터 확인
df['본거지법정동코드'].isnull().sum()

0

In [ ]:
df.shape

(25404385, 49)

In [ ]:
# about 39s
tot = df.merge(code, left_on='본거지법정동코드', right_on='법정동코드', how='left')
tot.shape

(25404385, 58)

In [ ]:
len(tot['차대번호'].unique())

25404384

In [ ]:
tot['차대번호'].isnull().sum()

0

# 전처리

## 말소제거
- 차량말소YN

In [ ]:
tot['차량말소YN'].value_counts(dropna=False)

In [ ]:
tot.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보', '법정동코드',
       '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역', '대기관리권역YN'],
      dtype='object')

## 폐지여부 확인

In [ ]:
tot['폐지여부'].value_counts(dropna=False)

존재     25322420
폐지        81963
NaN           2
Name: 폐지여부, dtype: int64

## 법정동코드 확인

In [ ]:
tot['법정동코드'].isnull().sum()

2

## 연료 확인

In [18]:
tot['연료'].isnull().sum()

264

In [19]:
tot['연료'].value_counts(dropna=False)

휘발유            12101061
경유              9860935
LPG(액화석유가스)     1896594
휘발유 하이브리드       1071375
전기               366804
경유 하이브리드          34191
CNG(압축천연가스)       30311
수소                28102
LPG 하이브리드         14350
CNG 하이브리드           304
NaN                 264
기타연료                 67
LNG(액화천연가스)          25
알코올                   2
Name: 연료, dtype: int64

### 기타연료의 차명으로 매칭

In [ ]:
tot.loc[tot['연료'] == '기타연료', '차명'].unique()

array(['홍성캠핑트레일러', '에어스트림탐지견수송트레일러'], dtype=object)

In [ ]:
# 홍성캠핑트레일러 : 무동력
# 에어스트림탐지견수송트레일러 : 무동력

In [ ]:
tot.loc[tot['차명'] == '홍성캠핑트레일러', '연료'] = '무동력'
tot.loc[tot['차명'] == '에어스트림탐지견수송트레일러', '연료'] = '무동력'

### 연료 결측값의 차명으로 매칭

In [ ]:
set(tot.loc[tot['연료'].isnull() == True, '차명'].unique())

{'BMW',
 'BMW 3-SERIES',
 'BMW 318I',
 'BMW 325CI',
 'BMW 325I',
 'BMW 328I',
 'BMW 525I',
 'BMW 528I',
 'BMW 530I',
 'BMW 5SERIES',
 'BMW X3',
 'BMW X5',
 'BMW Z3',
 'BMW530I',
 'BUICK CENTURY',
 'BUICK LESABRE',
 'BUICK TERRAZA',
 'CADILLAC CTS',
 'CHEVROLET IMPALA',
 'CHEVROLET PRIZM',
 'CHRYSLER',
 'CHRYSLER 300 LTD',
 'CHRYSLER 300C',
 'CHRYSLER HEMI AWD',
 'CHRYSLER PT CRUISER',
 'CHRYSLER TOWN & COUNTRY(7)',
 'CIELO',
 'CORSICA',
 'DODGE CALIBER',
 'DODGE CARAVAN',
 'DODGE CHARGER',
 'DODGE DAKOTA',
 'DODGE DURANGO',
 'DODGE GRAND CARAVAN',
 'DODGE NEON SXT',
 'FIREBIRD',
 'FORD ESCAPE',
 'FORD EXPEDITION',
 'FORD EXPEDITION(8)',
 'FORD EXPLORER',
 'FORD FOCUS',
 'FORD FREESTYLE(7)',
 'FORD FUSION',
 'FORD MUSTANG',
 'FORD RANGER XLT',
 'FORD WINDSTAR',
 'GMC JIMMY',
 'GMC SIERRA',
 'HONDA ACCORD',
 'HONDA CR-V',
 'HONDA CR-V EX',
 'HONDA PILOT',
 'HYUNDAI SONATA',
 'HYUNDAI/SONATA',
 'INFINITI G37S',
 'JAGUAR X-TYPE',
 'JEEP GRAND CHEROKEE',
 'JEEP LIBERTY',
 'JEEP PATRIOT',
 '

In [ ]:
# 'BMW' : 휘발유
# 'BMW 3-SERIES' : 휘발유
# 'BMW 318I' : 휘발유
# 'BMW 325CI' : 휘발유
# 'BMW 325I' : 휘발유
# 'BMW 328I' : 휘발유
# 'BMW 525I' : 휘발유
# 'BMW 528I' : 휘발유
# 'BMW 530I' : 휘발유
# 'BMW 5SERIES' : 휘발유
# 'BMW X3' : 휘발유
# 'BMW X5' : 휘발유
# 'BMW Z3' : 휘발유
# 'BMW530I' : 휘발유
# 'BUICK CENTURY' : 휘발유
# 'BUICK LESABRE' : 휘발유
# 'BUICK TERRAZA' : 휘발유
# 'CADILLAC CTS' : 휘발유
# 'CHEVROLET IMPALA' : 휘발유
# 'CHEVROLET PRIZM' : 휘발유
# 'CHRYSLER' : 휘발유
# 'CHRYSLER 300 LTD' : 휘발유
# 'CHRYSLER HEMI AWD' : 휘발유
# 'CHRYSLER PT CRUISER' : 휘발유
# 'CHRYSLER TOWN & COUNTRY(7)' : 휘발유
# 'CIELO' : 휘발유
# 'CORSICA' : 휘발유
# 'DODGE CALIBER' : 휘발유
# 'DODGE CARAVAN' : 휘발유
# 'DODGE CHARGER' : 휘발유
# 'DODGE DAKOTA' : 휘발유
# 'DODGE DURANGO' : 휘발유
# 'DODGE GRAND CARAVAN' : 휘발유
# 'DODGE NEON SXT' : 휘발유
# 'FIREBIRD' : 휘발유
# 'FORD ESCAPE' : 휘발유
# 'FORD EXPEDITION' : 휘발유
# 'FORD EXPEDITION(8)' : 휘발유
# 'FORD EXPLORER' : 휘발유
# 'FORD FOCUS' : 휘발유
# 'FORD FREESTYLE(7)' : 휘발유
# 'FORD FUSION' : 휘발유
# 'FORD MUSTANG' : 휘발유
# 'FORD WINDSTAR' : 휘발유
# 'GMC JIMMY' : 휘발유
# 'GMC SIERRA' : 휘발유
# 'HONDA ACCORD' : 휘발유
# 'HONDA CR-V' : 휘발유
# 'HONDA CR-V EX' : 휘발유
# 'HONDA PILOT' : 휘발유
# 'HYUNDAI SONATA' : 휘발유
# 'HYUNDAI/SONATA' : 휘발유
# 'INFINITI G37S' : 휘발유
# 'JAGUAR X-TYPE' : 휘발유
# 'JEEP GRAND CHEROKEE' : 휘발유
# 'JEEP LIBERTY' : 휘발유
# 'KIA OPTIMA' : 휘발유
# 'KIA SEDONA' : 휘발유
# 'LEXUS ES330' : 휘발유
# 'LEXUS ES350' : 휘발유
# 'LEXUS LS430' : 휘발유
# 'LEXUS RS330' : 휘발유
# 'LEXUS RX330' : 휘발유
# 'LINCOLN MKX' : 휘발유
# 'MAZDA MAZDA6' : 휘발유
# 'MAZDA SENTA' : 휘발유
# 'MERCEDES BENZ' : 휘발유
# 'MERCEDES-BENZ' : 휘발유
# 'MERCEDES-BENZ E-350' : 휘발유
# 'MERCEDES-BENZ E-CLASS' : 휘발유
# 'MERCEDES-BENZ S 500' : 휘발유
# 'MERCURY MILAN' : 휘발유
# 'NEW 그랜저XG' : 휘발유
# 'NISSAN MURANO' : 휘발유
# 'NISSAN ROGUE 3.5' : 휘발유
# 'NISSAN XTERRA' : 휘발유
# 'OLDSMOBILE BRAVADA' : 휘발유
# 'PLYMOUTH' : 휘발유
# 'PONTIAC SUNBIRD' : 휘발유
# 'PONTIAC VIBE' : 휘발유
# 'PONTIAC VIBE BASE' : 휘발유
# 'ROYALE' : 휘발유
# 'ROYALSALON' : 휘발유
# 'RX7' : 휘발유
# 'SANTAFE' : 휘발유
# 'SEPHIA' : 휘발유
# 'STAREX' : 휘발유
# 'SUNDANCE' : 휘발유
# 'TOYOTA AVALON' : 휘발유
# 'TOYOTA COROLLA' : 휘발유
# 'TOYOTA ESTIIM' : 휘발유
# 'TOYOTA MATRIX' : 휘발유
# 'TOYOTA SIENNA' : 휘발유
# 'TOYOTA SR-5' : 휘발유
# 'VOLKSWAGEN JETTA' : 휘발유
# 'VOLVO 960' : 휘발유
# 'VOLVO S60' : 휘발유
# 'VOLVO S80' : 휘발유
# 'VOLVO T5' : 휘발유
# 'VOLVO XC60' : 휘발유
# 'VOLVO XC90' : 휘발유
# '닛산씨마그로리아' : 휘발유
# '다찌람밴' : 휘발유
# '도오다' : 휘발유
# '도요다MR2' : 휘발유
# '벤츠 C300' : 휘발유
# '블루버드' : 휘발유
# '사파리' : 휘발유
# '새턴' : 휘발유
# '소나타1800오토매틱' : 휘발유
# '수프라' : 휘발유
# '시보레' : 휘발유
# '시보레카베리아' : 휘발유
# '쏘나타' : 휘발유
# '체로키' : 휘발유
# '코르시카' : 휘발유
# '코티나디럭스' : 휘발유
# '크로마터보i.e' : 휘발유
# '포니' : 휘발유
# '프레스토오토메틱' : 휘발유
# '프리무스' : 휘발유


In [ ]:
# 'JEEP PATRIOT' : 경유
# 'FORD RANGER XLT' : 경유
# 'CHRYSLER 300C' : 경유
# 'MERCEDES-BENZ CLS 550' : 경유
# 'MERCEDES-BENZ R-CLASS 3500' : 경유
# 'MERCEDES-BENZ S65' : 경유
# 'TOYOTA 4-RUNNER' : 경유
# '기아마스타' : 경유
# '뉴복사담프' : 경유
# '링컨MARK7' : 경유
# '새한2.5톤카고' : 경유
# '원흥식 트럭' : 경유
# '제설차' : 경유
# '지프' : 경유
# '현대3톤트럭' : 경유
# '현대포터' : 경유

In [ ]:
# '대흥저상트레일러' : 무동력
# '다임24KL에틸렌글리콜트레일러' : 무동력
# 홍성캠핑트레일러 : 무동력
# 에어스트림탐지견수송트레일러 : 무동력
# '한원40A콤비콘테이너트레일러' : 무동력
# '한원40피트구즈넥트레일러' : 무동력

In [ ]:
gas_list = ['BMW', 'BMW 3-SERIES', 'BMW 318I', 'BMW 325CI', 'BMW 325I', 'BMW 328I', 'BMW 525I',
'BMW 528I', 'BMW 530I', 'BMW 5SERIES', 'BMW X3', 'BMW X5', 'BMW Z3', 'BMW530I', 
'BUICK CENTURY', 'BUICK LESABRE', 'BUICK TERRAZA', 'CADILLAC CTS', 'CHEVROLET IMPALA',
'CHEVROLET PRIZM', 'CHRYSLER', 'CHRYSLER 300 LTD', 'CHRYSLER HEMI AWD', 
'CHRYSLER PT CRUISER', 'CHRYSLER TOWN & COUNTRY(7)', 'CIELO', 'CORSICA', 'DODGE CALIBER',
'DODGE CARAVAN', 'DODGE CHARGER', 'DODGE DAKOTA', 'DODGE DURANGO', 'DODGE GRAND CARAVAN',
'DODGE NEON SXT', 'FIREBIRD', 'FORD ESCAPE', 'FORD EXPEDITION', 'FORD EXPEDITION(8)',
'FORD EXPLORER', 'FORD FOCUS', 'FORD FREESTYLE(7)', 'FORD FUSION', 'FORD MUSTANG', 
'FORD RANGER XLT', 'FORD WINDSTAR', 'GMC JIMMY', 'GMC SIERRA', 'HONDA ACCORD', 
'HONDA CR-V', 'HONDA CR-V EX', 'HONDA PILOT', 'HYUNDAI SONATA', 'HYUNDAI/SONATA',
'INFINITI G37S', 'JAGUAR X-TYPE', 'JEEP GRAND CHEROKEE', 'JEEP LIBERTY', 'KIA OPTIMA',
'KIA SEDONA', 'LEXUS ES330', 'LEXUS ES350', 'LEXUS LS430', 'LEXUS RS330', 'LEXUS RX330',
'LINCOLN MKX', 'MAZDA MAZDA6', 'MAZDA SENTA', 'MERCEDES BENZ', 'MERCEDES-BENZ', 
'MERCEDES-BENZ E-350', 'MERCEDES-BENZ E-CLASS', 'MERCEDES-BENZ S 500', 'MERCEDES-BENZ S65',
'MERCURY MILAN', 'NEW 그랜저XG', 'NISSAN MURANO', 'NISSAN ROGUE 3.5', 'NISSAN XTERRA',
'OLDSMOBILE BRAVADA', 'PLYMOUTH', 'PONTIAC SUNBIRD', 'PONTIAC VIBE', 'PONTIAC VIBE BASE',
'ROYALE', 'ROYALSALON', 'RX7', 'SANTAFE', 'SEPHIA', 'STAREX', 'SUNDANCE', 'TOYOTA AVALON',
'TOYOTA COROLLA', 'TOYOTA ESTIIM', 'TOYOTA MATRIX', 'TOYOTA SIENNA', 'TOYOTA SR-5',
'VOLKSWAGEN JETTA', 'VOLVO 960', 'VOLVO S60', 'VOLVO S80', 'VOLVO T5', 'VOLVO XC60',
'VOLVO XC90', '닛산씨마그로리아', '다찌람밴', '도오다', '도요다MR2', '벤츠 C300', '블루버드',
'사파리', '새턴', '소나타1800오토매틱', '수프라', '시보레', '시보레카베리아', '쏘나타', 
'체로키', '코르시카', '코티나디럭스', '크로마터보i.e', '포니', '프레스토오토메틱', '프리무스',
'코티나마크IV', '브리샤2', '다이하스웨곤', '도요다크라운', '로얄살롱', '도요다코로나', 
'스텔라', '스텔라1.6CXL', '포니2', '도요다크라운로얄싸롱', '벤즈230E', '프레스토오토매틱', 
'르망펜타-5', '스텔라오토매틱', '엑셀', '벤즈 230E', '캐피탈', '로얄살롱2.0AT', '르망', 
'머큐리 세이블', '복사14', 'HYUNDAI PRESTO'
]
gas_hybrid_list = ['도요타 프리우스(하이브리드)', 'TOYOTA CAMRY하이브리드']
diesel_list = ['MERCEDES-BENZ R-CLASS 3500', 'JEEP PATRIOT', 'CHRYSLER 300C', 
'MERCEDES-BENZ CLS 550', 'TOYOTA 4-RUNNER', '기아마스타', '뉴복사담프', '링컨MARK7',
'새한2.5톤카고', '원흥식 트럭', '제설차', '지프', '현대3톤트럭', '현대포터', '히노탱크로리', 
'아시아AM707중형버스', '인타담프', '새한버스', '도요다웨곤엠브란스', '대우11톤카고', 
'대우8톤카고', '맥스디젤삼방', '현대리어엔진버스', '구급차(도요다)',
]
noengine_list = ['다임24KL에틸렌글리콜트레일러', '대흥저상트레일러',
'한원40A콤비콘테이너트레일러', '한원40피트구즈넥트레일러', '홍성캠핑트레일러', '에어스트림탐지견수송트레일러',
]

In [ ]:
for one in tqdm(gas_list):
    tot1.loc[tot1['차명'] == one, '연료'] = '휘발유'

100%|██████████| 119/119 [02:43<00:00,  1.37s/it]


In [ ]:
for one in tqdm(gas_hybrid_list):
    tot1.loc[tot1['차명'] == one, '연료'] = '휘발유 하이브리드'

In [ ]:
for one in tqdm(diesel_list):
    tot1.loc[tot1['차명'] == one, '연료'] = '경유'

100%|██████████| 14/14 [00:19<00:00,  1.37s/it]


In [ ]:
for one in tqdm(noengine_list):
    tot1.loc[tot1['차명'] == one, '연료'] = '무동력'

100%|██████████| 4/4 [00:05<00:00,  1.37s/it]


### 기타연료의 엔진형식으로 매칭

In [20]:
tot1.loc[tot1['연료'] == '기타연료', '엔진형식'].unique()

array(['.', '2AZ', '1NZ', '-', 'G4AJ', 'MSV3', 'B5', 'H20MR', 'G16SF',
       nan, 'G15CF', '01821300581', 'ZB', 'LNS', '09801220246', '3304124'],
      dtype=object)

In [94]:
tot1.loc[tot1['엔진형식'] == '2AZ', '연료'].value_counts(dropna=False)

경유           1336
휘발유 하이브리드       4
Name: 연료, dtype: int64

In [95]:
tot1.loc[tot1['엔진형식'] == '1NZ', '연료'].value_counts(dropna=False)

경유           2344
휘발유 하이브리드      12
Name: 연료, dtype: int64

In [96]:
tot1.loc[tot1['엔진형식'] == 'G4AJ', '연료'].value_counts(dropna=False)

휘발유    5054
Name: 연료, dtype: int64

In [97]:
tot1.loc[tot1['엔진형식'] == 'MSV3', '연료'].value_counts(dropna=False)

휘발유    1095
Name: 연료, dtype: int64

In [98]:
tot1.loc[tot1['엔진형식'] == 'B5', '연료'].value_counts(dropna=False)

휘발유    16111
Name: 연료, dtype: int64

In [102]:
tot1.loc[tot1['엔진형식'] == 'H20MR', '연료'].value_counts(dropna=False)

휘발유    2911
Name: 연료, dtype: int64

In [103]:
tot1.loc[tot1['엔진형식'] == 'G16SF', '연료'].value_counts(dropna=False)

휘발유    46
Name: 연료, dtype: int64

In [104]:
tot1.loc[tot1['엔진형식'] == 'G15CF', '연료'].value_counts(dropna=False)

휘발유    3248
Name: 연료, dtype: int64

In [105]:
tot1.loc[tot1['엔진형식'] == '01821300581', '연료'].value_counts(dropna=False)

휘발유    1
Name: 연료, dtype: int64

In [106]:
tot1.loc[tot1['엔진형식'] == 'ZB', '연료'].value_counts(dropna=False)

경유    5891
Name: 연료, dtype: int64

In [107]:
tot1.loc[tot1['엔진형식'] == 'LNS', '연료'].value_counts(dropna=False)

LPG(액화석유가스)    205
NaN              1
휘발유              1
Name: 연료, dtype: int64

In [108]:
tot1.loc[tot1['엔진형식'] == '09801220246', '연료'].value_counts(dropna=False)

경유    1
Name: 연료, dtype: int64

In [109]:
tot1.loc[tot1['엔진형식'] == '3304124', '연료'].value_counts(dropna=False)

기타연료    1
Name: 연료, dtype: int64

In [110]:
tot1.loc[tot1['엔진형식'] == '.', '연료'].value_counts(dropna=False)

기타연료    1
Name: 연료, dtype: int64

In [111]:
tot1.loc[tot1['엔진형식'] == '-', '연료'].value_counts(dropna=False)

기타연료    1
Name: 연료, dtype: int64

In [ ]:
# '2AZ' : 경유
# '1NZ' : 경유
# 'G4AJ' : 휘발유
# 'MSV3' : 휘발유
# 'B5' : 휘발유
# 'H20MR' : 휘발유
# 'G16SF' : 휘발유
# 'G15CF' : 휘발유
# '01821300581' : 휘발유
# 'ZB' : 경유
# 'LNS' : LPG(액화석유가스)
# '09801220246' : 경유
# '3304124' : 기타연료
# '.' : 기타연료
# '-' : 기타연료

In [114]:
tot1.loc[tot1['엔진형식'] == '2AZ', '연료'] = '경유'
tot1.loc[tot1['엔진형식'] == '1NZ', '연료'] = '경유'
tot1.loc[tot1['엔진형식'] == 'G4AJ', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == 'MSV3', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == 'B5', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == 'H20MR', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == 'G16SF', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == 'G15CF', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == '01821300581', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == 'ZB', '연료'] = '경유'
tot1.loc[tot1['엔진형식'] == 'LNS', '연료'] = 'LPG(액화석유가스)'
tot1.loc[tot1['엔진형식'] == '09801220246', '연료'] = '경유'
tot1.loc[tot1['엔진형식'] == '3304124', '연료'] = '휘발유'

### 연료 결측값의 엔진형식으로 매칭

In [115]:
tot1.loc[tot1['연료'].isnull() == True, '엔진형식'].unique()

array([nan, 'G4EC', 'K24A1', 'N62B48B', '0', 'G4JS', 'AZJ', 'G6BA', '5S',
       'G6CT', '2ZZ', 'JG1EK', '2JA', 'FE', '112 944', 'B230F', '119',
       'EV20', 'C32A1', '2JZ', '13B-REW', '3.8', 'V6', '5E', 'SR20',
       '2S25', 'M64/02', '117.986', '3S-GTE', '111 970', '104 941', 'EFA',
       'CRJ', 'M30-69', '5.0HD', 'B-123', 'F8C', '40S1', 'CR8.13.1',
       'G15MF', 'K04D3', 'VQ30', '102 962', 'mx', 'B', '31L', 'G4CP',
       '834C.146', 'G4AJF045570', 'G4MBG', 'GJS(LJS)', '04784300942',
       '10772217991', 'DA120', 'D0846HM', 'DA640'], dtype=object)

In [117]:
tot1.loc[tot1['엔진형식'] == 'G4EC', '연료'].value_counts(dropna=False)

휘발유    63813
NaN        1
Name: 연료, dtype: int64

In [118]:
tot1.loc[tot1['엔진형식'] == 'K24A1', '연료'].value_counts(dropna=False)

휘발유    1792
NaN       1
Name: 연료, dtype: int64

In [119]:
tot1.loc[tot1['엔진형식'] == 'N62B48B', '연료'].value_counts(dropna=False)

휘발유    1250
NaN       1
Name: 연료, dtype: int64

In [120]:
tot1.loc[tot1['엔진형식'] == '0', '연료'].value_counts(dropna=False)

휘발유          459
경유             6
휘발유 하이브리드      3
NaN            2
Name: 연료, dtype: int64

In [121]:
tot1.loc[tot1['엔진형식'] == 'G4JS', '연료'].value_counts(dropna=False)

휘발유    209
NaN      1
Name: 연료, dtype: int64

In [122]:
tot1.loc[tot1['엔진형식'] == 'AZJ', '연료'].value_counts(dropna=False)

휘발유    1036
NaN       1
Name: 연료, dtype: int64

In [123]:
tot1.loc[tot1['엔진형식'] == 'G6BA', '연료'].value_counts(dropna=False)

휘발유    6641
NaN       1
Name: 연료, dtype: int64

In [124]:
tot1.loc[tot1['엔진형식'] == '5S', '연료'].value_counts(dropna=False)

휘발유    52
NaN     1
Name: 연료, dtype: int64

In [125]:
tot1.loc[tot1['엔진형식'] == 'G6CT', '연료'].value_counts(dropna=False)

휘발유    14428
NaN        1
Name: 연료, dtype: int64

In [126]:
tot1.loc[tot1['엔진형식'] == '2ZZ', '연료'].value_counts(dropna=False)

휘발유    62
NaN     1
Name: 연료, dtype: int64

In [127]:
tot1.loc[tot1['엔진형식'] == 'JG1EK', '연료'].value_counts(dropna=False)

NaN    1
Name: 연료, dtype: int64

In [128]:
tot1.loc[tot1['엔진형식'] == '2JA', '연료'].value_counts(dropna=False)

NaN    1
Name: 연료, dtype: int64

In [129]:
tot1.loc[tot1['엔진형식'] == 'FE', '연료'].value_counts(dropna=False)

휘발유            13384
LPG(액화석유가스)     1395
NaN                1
Name: 연료, dtype: int64

In [130]:
tot1.loc[tot1['엔진형식'] == '112 944', '연료'].value_counts(dropna=False)

휘발유    433
NaN      1
Name: 연료, dtype: int64

In [131]:
tot1.loc[tot1['엔진형식'] == 'B230F', '연료'].value_counts(dropna=False)

휘발유    48
NaN     1
Name: 연료, dtype: int64

In [132]:
tot1.loc[tot1['엔진형식'] == '119', '연료'].value_counts(dropna=False)

휘발유    6
NaN    1
Name: 연료, dtype: int64

In [133]:
tot1.loc[tot1['엔진형식'] == 'EV20', '연료'].value_counts(dropna=False)

NaN    1
휘발유    1
Name: 연료, dtype: int64

In [134]:
tot1.loc[tot1['엔진형식'] == 'C32A1', '연료'].value_counts(dropna=False)

휘발유    3
NaN    1
Name: 연료, dtype: int64

In [135]:
tot1.loc[tot1['엔진형식'] == '2JZ', '연료'].value_counts(dropna=False)

휘발유    321
NaN      1
Name: 연료, dtype: int64

In [136]:
tot1.loc[tot1['엔진형식'] == '13B-REW', '연료'].value_counts(dropna=False)

NaN    1
Name: 연료, dtype: int64

In [137]:
tot1.loc[tot1['엔진형식'] == '3.8', '연료'].value_counts(dropna=False)

휘발유    72
NaN     2
Name: 연료, dtype: int64

In [138]:
tot1.loc[tot1['엔진형식'] == 'V6', '연료'].value_counts(dropna=False)

휘발유    9
NaN    2
Name: 연료, dtype: int64

In [139]:
tot1.loc[tot1['엔진형식'] == '5E', '연료'].value_counts(dropna=False)

휘발유    24
NaN     1
Name: 연료, dtype: int64

In [140]:
tot1.loc[tot1['엔진형식'] == 'SR20', '연료'].value_counts(dropna=False)

휘발유            113729
LPG(액화석유가스)     38883
NaN                 1
Name: 연료, dtype: int64

In [141]:
tot1.loc[tot1['엔진형식'] == '2S25', '연료'].value_counts(dropna=False)

NaN    1
Name: 연료, dtype: int64

In [142]:
tot1.loc[tot1['엔진형식'] == 'M64/02', '연료'].value_counts(dropna=False)

휘발유    3
NaN    1
Name: 연료, dtype: int64

In [143]:
tot1.loc[tot1['엔진형식'] == '117.986', '연료'].value_counts(dropna=False)

NaN    1
휘발유    1
Name: 연료, dtype: int64

In [144]:
tot1.loc[tot1['엔진형식'] == '3S-GTE', '연료'].value_counts(dropna=False)

NaN    1
Name: 연료, dtype: int64

In [145]:
tot1.loc[tot1['엔진형식'] == '111 970', '연료'].value_counts(dropna=False)

휘발유    118
NaN      1
Name: 연료, dtype: int64

In [146]:
tot1.loc[tot1['엔진형식'] == '104 941', '연료'].value_counts(dropna=False)

휘발유    11
NaN     1
Name: 연료, dtype: int64

In [147]:
tot1.loc[tot1['엔진형식'] == 'EFA', '연료'].value_counts(dropna=False)

NaN    1
Name: 연료, dtype: int64

In [148]:
tot1.loc[tot1['엔진형식'] == 'CRJ', '연료'].value_counts(dropna=False)

NaN    1
Name: 연료, dtype: int64

In [149]:
tot1.loc[tot1['엔진형식'] == 'M30-69', '연료'].value_counts(dropna=False)

휘발유    2
NaN    1
Name: 연료, dtype: int64

In [150]:
tot1.loc[tot1['엔진형식'] == '5.0HD', '연료'].value_counts(dropna=False)

NaN    1
Name: 연료, dtype: int64

In [151]:
tot1.loc[tot1['엔진형식'] == 'B-123', '연료'].value_counts(dropna=False)

NaN    1
Name: 연료, dtype: int64

In [152]:
tot1.loc[tot1['엔진형식'] == 'F8C', '연료'].value_counts(dropna=False)

휘발유    12183
NaN        1
Name: 연료, dtype: int64

In [153]:
tot1.loc[tot1['엔진형식'] == '40S1', '연료'].value_counts(dropna=False)

NaN    1
Name: 연료, dtype: int64

In [154]:
tot1.loc[tot1['엔진형식'] == 'CR8.13.1', '연료'].value_counts(dropna=False)

NaN    1
Name: 연료, dtype: int64

In [155]:
tot1.loc[tot1['엔진형식'] == 'G15MF', '연료'].value_counts(dropna=False)

휘발유    5332
NaN       1
Name: 연료, dtype: int64

In [156]:
tot1.loc[tot1['엔진형식'] == 'K04D3', '연료'].value_counts(dropna=False)

NaN    1
Name: 연료, dtype: int64

In [157]:
tot1.loc[tot1['엔진형식'] == 'VQ30', '연료'].value_counts(dropna=False)

휘발유    8
NaN    1
Name: 연료, dtype: int64

In [158]:
tot1.loc[tot1['엔진형식'] == '102 962', '연료'].value_counts(dropna=False)

휘발유    25
NaN     1
Name: 연료, dtype: int64

In [159]:
tot1.loc[tot1['엔진형식'] == 'mx', '연료'].value_counts(dropna=False)

NaN    1
Name: 연료, dtype: int64

In [160]:
tot1.loc[tot1['엔진형식'] == 'B', '연료'].value_counts(dropna=False)

휘발유    18447
NaN        1
Name: 연료, dtype: int64

In [161]:
tot1.loc[tot1['엔진형식'] == '31L', '연료'].value_counts(dropna=False)

NaN    1
Name: 연료, dtype: int64

In [162]:
tot1.loc[tot1['엔진형식'] == 'G4CP', '연료'].value_counts(dropna=False)

휘발유    40719
NaN        1
Name: 연료, dtype: int64

In [163]:
tot1.loc[tot1['엔진형식'] == '834C.146', '연료'].value_counts(dropna=False)

휘발유    18
NaN     1
Name: 연료, dtype: int64

In [164]:
tot1.loc[tot1['엔진형식'] == 'G4AJF045570', '연료'].value_counts(dropna=False)

NaN    1
Name: 연료, dtype: int64

In [165]:
tot1.loc[tot1['엔진형식'] == 'G4MBG', '연료'].value_counts(dropna=False)

NaN    1
Name: 연료, dtype: int64

In [166]:
tot1.loc[tot1['엔진형식'] == 'GJS(LJS)', '연료'].value_counts(dropna=False)

LPG(액화석유가스)    6
NaN            1
Name: 연료, dtype: int64

In [167]:
tot1.loc[tot1['엔진형식'] == '04784300942', '연료'].value_counts(dropna=False)

NaN    2
Name: 연료, dtype: int64

In [168]:
tot1.loc[tot1['엔진형식'] == '10772217991', '연료'].value_counts(dropna=False)

NaN    1
Name: 연료, dtype: int64

In [169]:
tot1.loc[tot1['엔진형식'] == 'DA120', '연료'].value_counts(dropna=False)

경유     3
NaN    1
Name: 연료, dtype: int64

In [170]:
tot1.loc[tot1['엔진형식'] == 'D0846HM', '연료'].value_counts(dropna=False)

경유     310
NaN      1
Name: 연료, dtype: int64

In [171]:
tot1.loc[tot1['엔진형식'] == 'DA640', '연료'].value_counts(dropna=False)

경유     3
NaN    1
Name: 연료, dtype: int64

In [176]:
# 'G4EC' : 휘발유
# 'K24A1' : 휘발유
# 'N62B48B' : 휘발유
# '0' : 휘발유
# 'G4JS' : 휘발유
# 'AZJ' : 휘발유
# 'G6BA' : 휘발유
# '5S' : 휘발유
# 'G6CT' : 휘발유
# '2ZZ' : 휘발유
# 'FE' : 휘발유
# '112 944' : 휘발유
# 'B230F' : 휘발유
# '119' : 휘발유
# 'EV20' : 휘발유
# 'C32A1' : 휘발유
# '2JZ' : 휘발유
# '3.8' : 휘발유
# 'V6' : 휘발유
# '5E' : 휘발유
# 'SR20' : 휘발유
# 'M64/02' : 휘발유
# '117.986' : 휘발유
# '111 970' : 휘발유
# '104 941' : 휘발유
# 'M30-69' : 휘발유
# 'F8C' : 휘발유
# 'G15MF' : 휘발유
# 'VQ30' : 휘발유
# '102 962' : 휘발유
# 'B' : 휘발유
# 'G4CP' : 휘발유
# '834C.146' : 휘발유
# 'GJS(LJS)' : LPG(액화석유가스)
# 'DA120' : 경유
# 'D0846HM' : 경유
# 'DA640' : 경유
# 'JG1EK' : nan
# '2JA' : nan
# '13B-REW' : nan
# '2S25' : nan
# '3S-GTE' : nan
# 'EFA' : nan
# 'CRJ' : nan
# '5.0HD' : nan
# 'B-123' : nan
# '40S1' : nan
# 'CR8.13.1' : nan
# 'K04D3' : nan
# 'mx' : nan
# '31L' : nan
# 'G4AJF045570' : nan
# 'G4MBG' : nan
# '04784300942' : nan
# '10772217991' : nan

In [172]:
tot1.loc[tot1['엔진형식'] == 'G4EC', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == 'K24A1', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == 'N62B48B', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == '0', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == 'G4JS', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == 'AZJ', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == 'G6BA', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == '5S', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == 'G6CT', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == '2ZZ', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == 'FE', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == '112 944', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == 'B230F', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == '119', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == 'EV20', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == 'C32A1', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == '2JZ', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == '3.8', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == 'V6', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == '5E', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == 'SR20', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == 'M64/02', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == '117.986', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == '111 970', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == '104 941', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == 'M30-69', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == 'F8C', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == 'G15MF', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == 'VQ30', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == '102 962', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == 'B', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == 'G4CP', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == '834C.146', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == 'GJS(LJS)', '연료'] = '휘발유'
tot1.loc[tot1['엔진형식'] == 'DA120', '연료'] = '경유'
tot1.loc[tot1['엔진형식'] == 'D0846HM', '연료'] = '경유'
tot1.loc[tot1['엔진형식'] == 'DA640', '연료'] = '경유'
# tot1.loc[tot1['엔진형식'] == 'JG1EK', '연료'] = np.nan
# tot1.loc[tot1['엔진형식'] == '2JA', '연료'] = np.nan
# tot1.loc[tot1['엔진형식'] == '13B-REW', '연료'] = np.nan
# tot1.loc[tot1['엔진형식'] == '2S25', '연료'] = np.nan
# tot1.loc[tot1['엔진형식'] == '3S-GTE', '연료'] = np.nan
# tot1.loc[tot1['엔진형식'] == 'EFA', '연료'] = np.nan
# tot1.loc[tot1['엔진형식'] == 'CRJ', '연료'] = np.nan
# tot1.loc[tot1['엔진형식'] == '5.0HD', '연료'] = np.nan
# tot1.loc[tot1['엔진형식'] == 'B-123', '연료'] = np.nan
# tot1.loc[tot1['엔진형식'] == '40S1', '연료'] = np.nan
# tot1.loc[tot1['엔진형식'] == 'CR8.13.1', '연료'] = np.nan
# tot1.loc[tot1['엔진형식'] == 'K04D3', '연료'] = np.nan
# tot1.loc[tot1['엔진형식'] == 'mx', '연료'] = np.nan
# tot1.loc[tot1['엔진형식'] == '31L', '연료'] = np.nan
# tot1.loc[tot1['엔진형식'] == 'G4AJF045570', '연료'] = np.nan
# tot1.loc[tot1['엔진형식'] == 'G4MBG', '연료'] = '휘발유'
# tot1.loc[tot1['엔진형식'] == '04784300942', '연료'] = np.nan
# tot1.loc[tot1['엔진형식'] == '10772217991', '연료'] = np.nan

In [173]:
tot1['연료'].value_counts(dropna=False)

휘발유            12140639
경유              9864640
LPG(액화석유가스)     1856311
휘발유 하이브리드       1068488
전기               366804
경유 하이브리드          34191
CNG(압축천연가스)       30311
수소                28102
LPG 하이브리드         14350
CNG 하이브리드           304
NaN                 216
LNG(액화천연가스)          25
기타연료                  2
알코올                   2
Name: 연료, dtype: int64

In [186]:
tot1.loc[tot1['차명'] == '도요다웨곤엠브란스', '자동차형식']

25397897    1980
Name: 자동차형식, dtype: object

In [187]:
tot1.loc[tot1['자동차형식'] == '1980', ['자동차형식', '연료', '차량연식']]

,자동차형식,연료,차량연식
25397897,1980,경유,2004


In [188]:
tot1.loc[tot1['차명'] == '구급차(도요다)', '자동차형식']

25369788    RX60
Name: 자동차형식, dtype: object

In [189]:
tot1.loc[tot1['자동차형식'] == 'RX60', ['자동차형식', '연료']]

,자동차형식,연료
25369788,RX60,경유


In [190]:
tot1.loc[tot1['차명'] == 'SID 500 F1', '자동차형식']

Series([], Name: 자동차형식, dtype: object)

In [191]:
tot1.loc[tot1['자동차형식'] == 'C1U52', ['자동차형식', '연료', '차명', ]]

,자동차형식,연료,차명


In [192]:
car_name = [
    '미스비시샤시(세미트레일러)', #
    '다이아몬드(추레라)피견인차', #
    '트레라', #
    '세미트레라', #
    '트레일러', #
    '세미트레일러', #
    '쎄미트레라', #
    '일산 육륜', 
    '히노 트레라', #
    '피견인차(탱크로리)', #
    '히노탱크로리트레라', #
    '히노탱크로리', # 경유
    '코티나마크IV', # 휘발유
    '후소트레일러', #
    '가와사끼샤시(세미트레일러)', #
    '도큐트레일러', #
    '세미 트레일러', #
    '콘테이너샤시콤바인', #
    '콘테이너샤시트레일러', #
    '인터내쇼날트레일러', #
    '추레라', #
    '프렛트홈트레일러콤바인', #
    '프렛트홈트레라콤바인', #
    '제너럴트레일러', #
    '아시아AM707중형버스', # 경유
    '훌러하프탱크추레라',  #
    '후루하프트레라', #
    '드라어  추레라', #
    '현대708콘테이너전용트레일러', #
    '인타담프', # 경유
    '동아저상식트레일러', #
    '저상식트레일러', #
    '브리샤2', # 휘발유
    '추레라(피견인차)', #
    '도큐사요트레러', #
    '새한버스', # 경유
    '다이하스웨곤', # 휘발유
    '스트리크트레일러샤시', #
    '프랫트홈트레일러콤바인', #
    '세미25톤트레일러', #
    '탱크로리트레라', #

    '도요다웨곤엠브란스', # 경유

    '도요다크라운', # 휘발유
    '추례라', #
    '진도콤비네이션콘테너샤시', #
    '쎄미트레일러', #
    '로얄살롱', # 휘발유
    '도요다코로나', # 휘발유
    '스텔라', # 휘발유
    '스텔라1.6CXL', # 휘발유
    '골드호퍼트레일러', #
    '대우11톤카고', # 경유
    '현대401드롭배드트레일러', #
    '대우8톤카고', # 경유
    '맥스디젤삼방', # 경유
    '현대리어엔진버스', # 경유
    '대우26톤드롭베스트레일러', #
    '포니2', # 휘발유
    '동아덤프트레일러', #
    '도요다크라운로얄싸롱', # 휘발유
    '현대구즈넥콘테이너트레일러', #
    '현대평판트레일러', #
    '현대20KL스텐탱크트레일러', #
    '23톤구즈넥덤프트레일러', #
    '구즈넥콘테이너샤시', #
    '26톤드톱베드트레일러', #

    '구급차(도요다)', # 경유

    '현대콘테이너트레일러',  #
    '동아콘테이너샤시', #
    '진도구즈넥콘테이너샤시', #
    '진도프레트폼트레일러', #
    '진도20피드콘테이저샤시', #
    '25M3벌크시멘트트레일러', #
    "현대45'익스텐션콘테이너트", #
    '벤즈230E', # 휘발유
    '진도구즈넥슬라이더콤비샤시', #
    '프레스토오토매틱', # 휘발유
    '르망펜타-5', # 휘발유
    '진도콤비네이션콘테이너샤시', #
    '스텔라오토매틱', # 휘발유
    '엑셀', # 휘발유
    '벤즈 230E', # 휘발유
    '진도구즈넥슬라이더콘테이너', #
    '동아콘테이너샤시콤바인', #
    '캐피탈', # 휘발유
    '로얄살롱2.0AT', # 휘발유
    '르망', # 휘발유
    '현대45`익스텐션콘테이너트', #
    '현대45,익스텐션콘테이너트', #
    '25M3벌크트레일러', #
    '23M3 벌크트레일러', #
    '머큐리 세이블', # 휘발유
    '트레일러  프렛홈형', # 
    '디엠티 탱크 트레일러', #
    '복사14', # 경유
    'HYUNDAI PRESTO', # 휘발유
    '미래40피트콤비네이션콘테이너샤시', #
    '도요타 프리우스(하이브리드)', # 휘발유 하이브리드
    'TOYOTA CAMRY하이브리드', # 휘발유 하이브리드
    '보트트레일러', # 
    '장보고 보트 트레일러', #
    '동우 가변형 평판트레일러', #
    'XW300', # 
    '대한평판트레일러', #
    '대한콤비컨테이너트레일러', #
    '바인스베르크400LK캠핑트레일러', #
    '미래40피트틸팅컨테이너샤시', #
    '소노마캠팡트레일러', #
    '에어스트림탐지견수송트레일러', #
    '이보1850캠핑트레일러', #
    'RW300', #
    'WSR레저장비운송트레일러', #
    '트레일마스터보트트레일러', #
    '제니스5캠핑트레일러', #
    'ALTEA542캠핑트레일러', #
    '트레일러-LMTV', #
    '카고 트레일러', #
    'TRAILER CARGO LMTV', #
    'D.K트레일러', #
    'FEISHEN일반화물운송트레일러', #

    'SID 500 F1', # 

    '카라반KUS2960평판트레일러', #
    '밤챔버용 트레일러', #
    '아우토니드이동가축관리트레일러', #
    '렉스온캠핑트레일러', #
    '포레스트리버캠핑트레일러', #
    '홍성푸드트레일러', #
    '펜트(FENDT)캠핑트레일러',#
    '밀폐형 방폭트레일러(DynaSEALR X10 LTR on Trailer)', #
    '홍성캠핑트레일러', #
    '아드리아(ADRIA)캠핑트레일러', #
    '트리가노(TRIGANO)스탈렛캠핑트레일러', #
    '엘디스(ELDDIS)캠핑트레일러',#
    '베일리(BAILEY)캠핑트레일러', #
    '바인스버그(WEINSBERG) 캠핑트레일러', #
    '미래40피트평판트레일러', #
    '타버트(TABBERT)캠핑트레일러', #
    '씨티인터내셔널캠핑트레일러', #
]
len(car_name)

138

In [199]:
# 47s
tot1.loc[tot1['차명'] == '코티나마크IV', '연료'] = '휘발유'
tot1.loc[tot1['차명'] == '브리샤2', '연료'] = '휘발유'
tot1.loc[tot1['차명'] == '다이하스웨곤', '연료'] = '휘발유'
tot1.loc[tot1['차명'] == '도요다크라운', '연료'] = '휘발유'
tot1.loc[tot1['차명'] == '로얄살롱', '연료'] = '휘발유'
tot1.loc[tot1['차명'] == '도요다코로나', '연료'] = '휘발유'
tot1.loc[tot1['차명'] == '스텔라', '연료'] = '휘발유'
tot1.loc[tot1['차명'] == '스텔라1.6CXL', '연료'] = '휘발유'
tot1.loc[tot1['차명'] == '포니2', '연료'] = '휘발유'
tot1.loc[tot1['차명'] == '도요다크라운로얄싸롱', '연료'] = '휘발유'
tot1.loc[tot1['차명'] == '벤즈230E', '연료'] = '휘발유'
tot1.loc[tot1['차명'] == '프레스토오토매틱', '연료'] = '휘발유'
tot1.loc[tot1['차명'] == '르망펜타-5', '연료'] = '휘발유'
tot1.loc[tot1['차명'] == '스텔라오토매틱', '연료'] = '휘발유'
tot1.loc[tot1['차명'] == '엑셀', '연료'] = '휘발유'
tot1.loc[tot1['차명'] == '벤즈 230E', '연료'] = '휘발유'
tot1.loc[tot1['차명'] == '캐피탈', '연료'] = '휘발유'
tot1.loc[tot1['차명'] == '로얄살롱2.0AT', '연료'] = '휘발유'
tot1.loc[tot1['차명'] == '르망', '연료'] = '휘발유'
tot1.loc[tot1['차명'] == '머큐리 세이블', '연료'] = '휘발유'
tot1.loc[tot1['차명'] == '복사14', '연료'] = '경유'
tot1.loc[tot1['차명'] == 'HYUNDAI PRESTO', '연료'] = '휘발유'

tot1.loc[tot1['차명'] == '도요타 프리우스(하이브리드)', '연료'] = '휘발유 하이브리드'
tot1.loc[tot1['차명'] == 'TOYOTA CAMRY하이브리드', '연료'] = '휘발유 하이브리드'

tot1.loc[tot1['차명'] == '히노탱크로리', '연료'] = '경유'
tot1.loc[tot1['차명'] == '아시아AM707중형버스', '연료'] = '경유'
tot1.loc[tot1['차명'] == '인타담프', '연료'] = '경유'
tot1.loc[tot1['차명'] == '새한버스', '연료'] = '경유'
tot1.loc[tot1['차명'] == '도요다웨곤엠브란스', '연료'] = '경유'
tot1.loc[tot1['차명'] == '대우11톤카고', '연료'] = '경유'
tot1.loc[tot1['차명'] == '대우8톤카고', '연료'] = '경유'
tot1.loc[tot1['차명'] == '맥스디젤삼방', '연료'] = '경유'
tot1.loc[tot1['차명'] == '현대리어엔진버스', '연료'] = '경유'
tot1.loc[tot1['차명'] == '구급차(도요다)', '연료'] = '경유'

In [200]:
tot1['연료'].value_counts(dropna=False)

휘발유            12140895
경유              9864585
LPG(액화석유가스)     1856308
휘발유 하이브리드       1068490
전기               366804
경유 하이브리드          34191
CNG(압축천연가스)       30311
수소                28102
LPG 하이브리드         14350
CNG 하이브리드           304
LNG(액화천연가스)          25
NaN                  11
무동력                   7
알코올                   2
Name: 연료, dtype: int64

### 최종 확인

In [ ]:
tot1.loc[tot1['연료'].isnull() == True, ['차명', '연료', '엔진형식', '자동차형식']]

In [205]:
# 12s
# issue 이미 위에서 처리한 차명의 연료가 결측값으로 나옴
tot1.loc[tot1['차명'] == 'LINCOLN MKX', '연료'] = '휘발유'
tot1.loc[tot1['차명'] == 'LEXUS RX330', '연료'] = '휘발유'
tot1.loc[tot1['차명'] == 'TOYOTA AVALON', '연료'] = '휘발유'
tot1.loc[tot1['차명'] == 'MERCEDES-BENZ S65', '연료'] = '휘발유'
tot1.loc[tot1['차명'] == 'MERCURY MILAN', '연료'] = '휘발유'
tot1.loc[tot1['차명'] == 'PONTIAC VIBE BASE', '연료'] = '휘발유'
tot1.loc[tot1['차명'] == 'TOYOTA COROLLA', '연료'] = '휘발유'
tot1.loc[tot1['차명'] == 'ROYALE', '연료'] = '휘발유'

In [206]:
tot1['연료'].value_counts(dropna=False)

휘발유            12140911
경유              9864582
LPG(액화석유가스)     1856308
휘발유 하이브리드       1068488
전기               366804
경유 하이브리드          34191
CNG(압축천연가스)       30311
수소                28102
LPG 하이브리드         14350
CNG 하이브리드           304
LNG(액화천연가스)          25
무동력                   7
알코올                   2
Name: 연료, dtype: int64

## 임시등급 병합

In [207]:
tot1['배출가스등급'].value_counts(dropna=False)

2     10960196
3      8623093
1      2771380
4      1511174
5      1154695
T2      247481
T3      136300
T4          37
T5          29
Name: 배출가스등급, dtype: int64

In [211]:
tot1.shape

(25404385, 59)

In [212]:
grade_list = []
for one in tqdm(tot1['배출가스등급']):
    if str(one) == '1':
        grade_list.append('1')
    elif str(one) == '2' or str(one) == 'T2':
        grade_list.append('2')
    elif str(one) == '3' or str(one) == 'T3':
        grade_list.append('3')
    elif str(one) == '4' or str(one) == 'T4':
        grade_list.append('4')
    elif str(one) == '5' or str(one) == 'T5':
        grade_list.append('5')
    elif str(one) == 'X':
        grade_list.append('X')
    else:
        grade_list.append(np.nan)
len(grade_list)

25404385

In [213]:
grade_list[:5]

['1', '1', '1', '1', '1']

In [214]:
tot1['등급'] = grade_list

## 연료 결측값 샘플

In [215]:
# about 28s
total = tot1.copy()

In [216]:
total[total['연료'].isnull() == True].shape

(0, 59)

In [34]:
total.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보', '법정동코드',
       '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역', '대기관리권역YN', '등급'],
      dtype='object')

In [35]:
total.loc[total['연료'].isnull() == True, ['차명', '엔진형식', '차량연식']].drop_duplicates(['차명', '엔진형식', '차량연식'])

,차명,엔진형식,차량연식
11998429,CHRYSLER TOWN & COUNTRY(7),NaN,2010
12045462,FORD ESCAPE,NaN,2010
12048481,JEEP GRAND CHEROKEE,NaN,2010
12073761,MERCEDES-BENZ E-CLASS,NaN,2010
12076693,VOLVO XC60,NaN,2010
...,...,...,...
25321981,GMC,DA120,1976
25322123,기아마스타,NaN,1976
25322257,코티나디럭스,NaN,1974
25322377,GMC,D0846HM,1967


#### [출력] 연료 결측값 샘플(차명, 엔진형식, 차량연식)

In [ ]:
# total.loc[total['연료'].isnull() == True, ['차명', '엔진형식', '차량연식']].drop_duplicates(['차명', '엔진형식']).to_excel('analysis/샘플_연료_결측값_차명_엔진형식_차량연식.xlsx', index=False)

## 연료 결측치 수정

### [로드] 샘플_연료_결측값_차명_엔진형식_차량연식(연료수정)

In [36]:
corrected_fold = 'D:/data/low_gas_vehicle'

In [37]:
corrected_fuel = pd.read_excel(os.path.join(corrected_fold, '샘플_연료_결측값_차명_엔진형식_차량연식(연료수정).xlsx'))
corrected_fuel.shape

(183, 4)

In [38]:
corrected_fuel.head()

,차명,연료,엔진형식,차량연식
0,CHRYSLER TOWN & COUNTRY(7),휘발유,NaN,2010
1,FORD ESCAPE,휘발유,NaN,2010
2,JEEP GRAND CHEROKEE,휘발유,NaN,2010
3,MERCEDES-BENZ E-CLASS,휘발유,NaN,2010
4,VOLVO XC60,휘발유,NaN,2010


In [39]:
corrected_fuel_dict = {a:b for a, b in zip(corrected_fuel['차명'], corrected_fuel['연료'])}

In [40]:
empty_fuel_car_idx = total[total['연료'].isnull() == True].index

In [41]:
len(empty_fuel_car_idx)

257

In [42]:
for i in tqdm(empty_fuel_car_idx):
    temp_car_name = total.loc[i, '차명']
    if temp_car_name in corrected_fuel_dict.keys():
        total.loc[i, '연료'] = corrected_fuel_dict[temp_car_name]

100%|██████████| 257/257 [00:22<00:00, 11.51it/s]


In [43]:
total[total['연료'].isnull() == True].shape

(0, 59)

## 기타 연료 확인

In [217]:
total.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보', '법정동코드',
       '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역', '대기관리권역YN', '등급'],
      dtype='object')

In [218]:
total.loc[total['연료'] == '기타연료', ['배출가스등급', '차명', '자동차형식','엔진형식', '차량연식', '연료', '시도', '시구군', '읍면동', '리', '지역3']]

,배출가스등급,차명,자동차형식,엔진형식,차량연식,연료,시도,시구군,읍면동,리,지역3


In [70]:
total.loc[total['차명'] == '홍성캠핑트레일러', '연료'] = '무동력'
total.loc[total['차명'] == '에어스트림탐지견수송트레일러', '연료'] = '무동력'
total.loc[total['차명'] == '일산 육륜', '연료'] = '경유'

In [71]:
total.loc[total['연료'] == '기타연료'].shape

(0, 59)

## 본거지법정동코드 확인
- 1대 : 정기, 정밀 검사 데이터(~22.06.30) 없음

In [219]:
total['시구군'].unique()[:5]

array(['청주시 상당구', '천안시 동남구', '구로구', '군산시', '창원시 성산구'], dtype=object)

In [220]:
total.loc[total['대기관리권역YN'].isnull() == True, ['본거지법정동코드', '시도', '시구군', '읍면동', '리', '지역3', '차량말소YN', '차량연식', '배출가스등급', '차대번호', '폐지여부', '대기관리권역']]

,본거지법정동코드,시도,시구군,읍면동,리,지역3,차량말소YN,차량연식,배출가스등급,차대번호,폐지여부,대기관리권역
24137513,4163055000,NaN,NaN,NaN,NaN,NaN,N,1992,4,KMHVE21NPNU122861,NaN,NaN
24922881,4165052000,NaN,NaN,NaN,NaN,NaN,N,2002,5,KMFGA17AP2C167888,NaN,NaN
25398594,4100000000,경기,NaN,NaN,NaN,NaN,N,1974,5,10742255732,존재,수도권


In [221]:
total.loc[total['본거지법정동코드'] == 4163055000, ['시도', '시구군', '대기관리권역YN', '대기관리권역']] = ['경기', '양주시', 'Y', '수도권']
total.loc[total['본거지법정동코드'] == 4165052000, ['시도', '시구군', '대기관리권역YN', '대기관리권역']] = ['경기', '포천시', 'Y', '수도권']
total.loc[total['본거지법정동코드'] == 4100000000, ['시도', '대기관리권역YN']] = ['경기', 'Y']

In [222]:
total.loc[total['대기관리권역YN'].isnull() == True].shape

(0, 59)

In [246]:
total.shape

(25404385, 59)

# [출력] 등록&제원정보(연료수정)(등급컬럼추가)

In [247]:
# about 11m 23s
total.to_csv(os.path.join(ods_fold, '[ODS]등록정보&제원정보(연료수정)(등급컬럼추가)(2022.12.28).csv'), index=False)

In [248]:
# about 6s
g4 = total[total['등급'] == '4'].reset_index(drop=True)
g5 = total[total['등급'] == '5'].reset_index(drop=True)
g4.shape, g5.shape

((1511211, 59), (1154724, 59))

In [249]:
# about 1m 9s
g4.to_csv(os.path.join(ods_fold, '[ODS][G4]등록정보&제원정보(연료수정)(등급컬럼추가)(2022.12.28).csv'), index=False)
g5.to_csv(os.path.join(ods_fold, '[ODS][G5]등록정보&제원정보(연료수정)(등급컬럼추가)(2022.12.28).csv'), index=False)

# [로드] 등록&제원정보(연료수정)(등급컬럼추가)

In [24]:
# about 7m 42s
total = pd.read_csv(os.path.join(ods_fold, '[ODS]등록정보&제원정보(연료수정)(등급컬럼추가)(2022.12.28).csv'), low_memory=False)

In [4]:
g5 = pd.read_csv(os.path.join(ods_fold, '[ODS][G5]등록정보&제원정보(연료수정)(등급컬럼추가)(2022.12.28).csv'), low_memory=False)

# 분석

##  연료별 통계

In [25]:
total.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보', '법정동코드',
       '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역', '대기관리권역YN', '등급'],
      dtype='object')

In [26]:
total.groupby(['연료', '등급'], dropna=False)['차대번호'].count()

연료           등급
CNG 하이브리드    1         287
             2          17
CNG(압축천연가스)  2       24307
             3        5998
             4           1
             5           5
LNG(액화천연가스)  2          25
LPG 하이브리드    1       14349
             2           1
LPG(액화석유가스)  1      461745
             2     1171739
             3      197701
             4       24692
             5         431
경유           1        2865
             2         485
             3     7584349
             4     1152694
             5     1124189
경유 하이브리드     3       34191
무동력          3           1
             4           1
             5           5
수소           1       28102
알코올          4           1
             5           1
전기           1      366804
휘발유          1      969107
             2     9870738
             3      937151
             4      333822
             5       30093
휘발유 하이브리드    1      928121
             2      140365
             3           2
Name: 차대번호, dtype: int64

In [27]:
total['등급'].isnull().sum()

0

In [28]:
total.groupby(['연료', '등급'], dropna=False)['차대번호'].count().unstack()

등급,1,2,3,4,5
연료,,,,,
CNG 하이브리드,287.0,17.0,NaN,NaN,NaN
CNG(압축천연가스),NaN,24307.0,5998.0,1.0,5.0
LNG(액화천연가스),NaN,25.0,NaN,NaN,NaN
LPG 하이브리드,14349.0,1.0,NaN,NaN,NaN
LPG(액화석유가스),461745.0,1171739.0,197701.0,24692.0,431.0
경유,2865.0,485.0,7584349.0,1152694.0,1124189.0
경유 하이브리드,NaN,NaN,34191.0,NaN,NaN
무동력,NaN,NaN,1.0,1.0,5.0
수소,28102.0,NaN,NaN,NaN,NaN


## 5등급 경유 대기관리권역별 통계

In [227]:
g5 = total[total['등급'] == '5'].reset_index(drop=True)
g5.shape

(1154724, 59)

### 5등급 경유차만 추출

In [6]:
g5_diesel = g5[g5['연료'] == '경유'].reset_index(drop=True)
g5_diesel.shape

(1124189, 59)

In [7]:
g5_diesel.groupby(['대기관리권역', '시도'], dropna=False)['차대번호'].count()

대기관리권역  시도
그 외     강원     47920
        제주     24541
남부권     광주     25657
        전남     76689
동남권     경남    103736
        경북    121048
        대구     48997
        부산     65654
        울산     25880
수도권     경기    222434
        서울    116755
        인천     53407
중부권     대전     25750
        세종      4272
        전북     57593
        충남     61964
        충북     41892
Name: 차대번호, dtype: int64

In [8]:
g5_diesel['시도'].isnull().sum()

0

In [9]:
g5_diesel['대기관리권역'].value_counts(dropna=False)

수도권    392596
동남권    365315
중부권    191471
남부권    102346
그 외     72461
Name: 대기관리권역, dtype: int64

In [10]:
g5_diesel['대기관리권역YN'].value_counts(dropna=False)

Y    843858
N    280331
Name: 대기관리권역YN, dtype: int64

In [11]:
g5_diesel.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보', '법정동코드',
       '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역', '대기관리권역YN', '등급'],
      dtype='object')

In [13]:
g5_y = g5_diesel[g5_diesel['대기관리권역YN'] == 'Y']
g5_y.shape

(843858, 59)

In [14]:
g5_n = g5_diesel[g5_diesel['대기관리권역YN'] == 'N']
g5_n.shape

(280331, 59)

In [15]:
g5_y.groupby(['대기관리권역', '시도'], dropna=False)['차대번호'].count()

대기관리권역  시도
남부권     광주     25657
        전남     32613
동남권     경남     60763
        경북     55056
        대구     48997
        부산     65654
        울산     25880
수도권     경기    213339
        서울    116755
        인천     52217
중부권     대전     25750
        세종      4272
        전북     26201
        충남     59159
        충북     31545
Name: 차대번호, dtype: int64

In [16]:
g5_n.groupby(['대기관리권역', '시도'], dropna=False)['차대번호'].count()

대기관리권역  시도
그 외     강원    47920
        제주    24541
남부권     전남    44076
동남권     경남    42973
        경북    65992
수도권     경기     9095
        인천     1190
중부권     전북    31392
        충남     2805
        충북    10347
Name: 차대번호, dtype: int64

### [출력] 연료별 통계, 5등급 대기관리권역별 통계

In [35]:
today_date = datetime.today().strftime("%Y.%m.%p_%H_%M_%S")
today_date

'2023.02.AM_08_26_54'

In [36]:
with pd.ExcelWriter(f'analysis/통계_연료별_5등급대기관리권역별{today_date}.xlsx') as writer:
    total.groupby(['연료', '등급'], dropna=False)['차대번호'].count().unstack().to_excel(writer, sheet_name='연료별')
    g5_y.groupby(['대기관리권역', '시도'], dropna=False)['차대번호'].count().to_excel(writer, sheet_name='대기관리권역별_y')
    g5_n.groupby(['대기관리권역', '시도'], dropna=False)['차대번호'].count().to_excel(writer, sheet_name='대기관리권역별_n')

## 4, 5등급 검사 기록없는 차량 수

## [로드] 정기&정밀검사

In [244]:
# about 4m 21s
file_name = '[ODS]정기&정밀검사(최신검사만)(2022.12.28).csv'
tin = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
tin.info()

MemoryError: Unable to allocate 168. MiB for an array with shape (22044208,) and data type int64

In [ ]:
tin.columns

Index(['사용연료', '주행거리', '차명', '차량번호', '차대번호', '차종', '차량연식', '엔진형식', '검사방법',
       '검사일자', '검사종류', '검사판정시각', '검사판정', '산소값', '이산화탄소값', '무부하매연판정1',
       '무부하매연판정2', '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6', '무부하매연허용치1',
       '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5', '무부하매연허용치6',
       '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5',
       '무부하매연측정치6', '차량용도', '최대출력허용치', '최대출력측정값', '검사접수번호', '검사소명', '재검사기간',
       '정기정밀', '검사 검사소코드', '검사유효기간'],
      dtype='object')

## 최근 3년 검사 데이터 추출

In [ ]:
tin['검사일자'].describe()

count   22044208.00
mean    20213557.70
std         8679.78
min     20190102.00
25%     20210427.00
50%     20211222.00
75%     20220614.00
max     20230107.00
Name: 검사일자, dtype: float64

In [ ]:
tin1 = tin[(tin['검사일자'] >= 20200101) & (tin['검사일자'] <= 20221231)]
tin1.shape

(20547872, 42)

In [ ]:
tin1['검사일자'].describe()

count   20547872.00
mean    20214858.24
std         6580.17
min     20200102.00
25%     20210611.00
50%     20220113.00
75%     20220620.00
max     20221231.00
Name: 검사일자, dtype: float64

In [ ]:
g4 = total[total['등급'] == '4'].reset_index(drop=True)
g4.shape

In [ ]:
g5.shape

In [ ]:
g4['연료'].value_counts(dropna=False)

경유             1152697
휘발유             332313
LPG(액화석유가스)      26089
NaN                 71
기타연료                39
CNG(압축천연가스)          1
알코올                  1
Name: 연료, dtype: int64

In [ ]:
g5['연료'].value_counts(dropna=False)

경유             1124188
휘발유              30040
LPG(액화석유가스)        436
NaN                 29
기타연료                25
CNG(압축천연가스)          5
알코올                  1
Name: 연료, dtype: int64

In [ ]:
g4.shape

(1511251, 49)

In [ ]:
g4 = g4.sort_values(['최초등록일자', '배출가스검사유효일자'], ascending=[False, False]).drop_duplicates('차대번호').reset_index(drop=True)
g4.shape

(1511211, 49)

In [ ]:
g5.shape

(1154762, 49)

In [ ]:
g5 = g5.sort_values(['최초등록일자', '배출가스검사유효일자'], ascending=[False, False]).drop_duplicates('차대번호').reset_index(drop=True)
g5.shape

(1154724, 49)

In [ ]:
# about 47s
g4_tin = g4.merge(tin1, on='차대번호', how='left')
g4_tin.shape

(1511211, 90)

In [ ]:
# about 43s
g5_tin = g5.merge(tin1, on='차대번호', how='left')
g5_tin.shape

(1154724, 90)

In [ ]:
g4_tin['검사방법'].unique()

array([nan, '부하검사(KD-147)', '부하검사(LUG DOWN)', '무부하검사(급가속)',
       '부하검사(ASM-Idling)', '무부하검사(TSI)', '무부하검사(정지가동)'], dtype=object)

In [ ]:
g4_tin['검사종류'].unique()

array([nan, '2부재검사(종합)', '종합검사(경과)', '종합검사', '정기검사', '정기(경과)검사',
       '재검사(2부재검사)', '재검사(1부재검사)', '1부재검사(종합)'], dtype=object)

In [ ]:
g5_tin['검사방법'].unique()

array([nan, '부하검사(KD-147)', '무부하검사(급가속)', '부하검사(LUG DOWN)', '무부하검사(TSI)',
       '부하검사(ASM-Idling)', '무부하검사(정지가동)'], dtype=object)

In [ ]:
g5_tin['검사종류'].unique()

array([nan, '종합검사', '2부재검사(종합)', '정기검사', '종합검사(경과)', '재검사(2부재검사)',
       '재검사(1부재검사)', '1부재검사(종합)', '정기(경과)검사'], dtype=object)

In [ ]:
view_cols = ['검사방법', '검사일자', '검사종류', '검사판정시각', '검사판정', 
'무부하매연판정1', '무부하매연측정치1',
'무부하매연판정2', '무부하매연측정치2',
'무부하매연판정3', '무부하매연측정치3',
'무부하매연판정4', '무부하매연측정치4',
'무부하매연판정5', '무부하매연측정치5',
'무부하매연판정6', '무부하매연측정치6',]

In [ ]:
g4_tin[g4_tin['연료'] == '알코올'].reset_index(drop=True)[view_cols].isnull().sum().reset_index()

,index,0
0,검사방법,0
1,검사일자,0
2,검사종류,0
3,검사판정시각,0
4,검사판정,0
5,무부하매연판정1,0
6,무부하매연측정치1,0
7,무부하매연판정2,0
8,무부하매연측정치2,0
9,무부하매연판정3,0


In [ ]:
g4_miss = pd.DataFrame({'컬럼명':view_cols})
for one in g4_tin['연료'].unique():
    temp = g4_tin[g4_tin['연료'] == one].reset_index(drop=True)[view_cols].isnull().sum().reset_index()
    temp.columns = ['컬럼명', f'4등급_{one}']
    g4_miss = g4_miss.merge(temp, on='컬럼명', how='left')

In [ ]:
g4_miss

,컬럼명,4등급_경유,4등급_휘발유,4등급_nan,4등급_기타연료,4등급_LPG(액화석유가스),4등급_CNG(압축천연가스),4등급_알코올
0,검사방법,44043,275254,0.00,37,21761,0,0
1,검사일자,44043,275254,0.00,37,21761,0,0
2,검사종류,44043,275254,0.00,37,21761,0,0
3,검사판정시각,52620,275825,0.00,37,21806,0,0
4,검사판정,44043,275254,0.00,37,21761,0,0
5,무부하매연판정1,52625,276357,0.00,38,21859,0,0
6,무부하매연측정치1,52620,276357,0.00,38,21859,0,0
7,무부하매연판정2,1108848,276368,0.00,38,21859,1,0
8,무부하매연측정치2,794724,276366,0.00,38,21859,1,0
9,무부하매연판정3,1109728,276368,0.00,38,21859,1,0


In [ ]:
g5_miss = pd.DataFrame({'컬럼명':view_cols})
for one in g5_tin['연료'].unique():
    temp = g5_tin[g5_tin['연료'] == one].reset_index(drop=True)[view_cols].isnull().sum().reset_index()
    temp.columns = ['컬럼명', f'5등급_{one}']
    g5_miss = g5_miss.merge(temp, on='컬럼명', how='left')

In [ ]:
g5_miss

,컬럼명,5등급_경유,5등급_휘발유,5등급_CNG(압축천연가스),5등급_LPG(액화석유가스),5등급_nan,5등급_기타연료,5등급_알코올
0,검사방법,417776,29670,3,433,0.00,25,1
1,검사일자,417776,29670,3,433,0.00,25,1
2,검사종류,417776,29670,3,433,0.00,25,1
3,검사판정시각,423727,29674,3,433,0.00,25,1
4,검사판정,417776,29670,3,433,0.00,25,1
5,무부하매연판정1,423936,29681,3,433,0.00,25,1
6,무부하매연측정치1,423927,29681,3,433,0.00,25,1
7,무부하매연판정2,1017450,29683,3,433,0.00,25,1
8,무부하매연측정치2,890184,29683,3,433,0.00,25,1
9,무부하매연판정3,1017776,29683,3,433,0.00,25,1


In [ ]:
g45_miss = g4_miss.merge(g5_miss, on='컬럼명', how='left')
g45_miss

,컬럼명,4등급_경유,4등급_휘발유,4등급_nan,4등급_기타연료,4등급_LPG(액화석유가스),4등급_CNG(압축천연가스),4등급_알코올,5등급_경유,5등급_휘발유,5등급_CNG(압축천연가스),5등급_LPG(액화석유가스),5등급_nan,5등급_기타연료,5등급_알코올
0,검사방법,44043,275254,0.00,37,21761,0,0,417776,29670,3,433,0.00,25,1
1,검사일자,44043,275254,0.00,37,21761,0,0,417776,29670,3,433,0.00,25,1
2,검사종류,44043,275254,0.00,37,21761,0,0,417776,29670,3,433,0.00,25,1
3,검사판정시각,52620,275825,0.00,37,21806,0,0,423727,29674,3,433,0.00,25,1
4,검사판정,44043,275254,0.00,37,21761,0,0,417776,29670,3,433,0.00,25,1
5,무부하매연판정1,52625,276357,0.00,38,21859,0,0,423936,29681,3,433,0.00,25,1
6,무부하매연측정치1,52620,276357,0.00,38,21859,0,0,423927,29681,3,433,0.00,25,1
7,무부하매연판정2,1108848,276368,0.00,38,21859,1,0,1017450,29683,3,433,0.00,25,1
8,무부하매연측정치2,794724,276366,0.00,38,21859,1,0,890184,29683,3,433,0.00,25,1
9,무부하매연판정3,1109728,276368,0.00,38,21859,1,0,1017776,29683,3,433,0.00,25,1


In [ ]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%H.%M.%S")
today_date

'2023.02.22_PM.15.33.36'

In [ ]:
g45_miss.to_excel(f'analysis2/4_5등급_검사_결측치_{today_date}.xlsx', index=False)

# 코드 마지막